In [70]:
import re
import string
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize


In [71]:
# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdul.muhmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdul.muhmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdul.muhmin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\abdul.muhmin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [72]:
# Load dataset
def load_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read().lower()
    return text

In [73]:
# Function to map POS tags to WordNet format
def get_wordnet_pos(nltk_pos):
    if nltk_pos.startswith('J'):
        return wordnet.ADJ
    elif nltk_pos.startswith('V'):
        return wordnet.VERB
    elif nltk_pos.startswith('N'):
        return wordnet.NOUN
    elif nltk_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default noun


In [74]:
# Tokenization and Sequence Generation
def preprocess_text(text, sequence_length=5):

    
    # Remove special characters using regex
    text = re.sub(r"[^\w\s]", "", text)  # Keep only words and spaces

    # Tokenization
    tokenized_words = word_tokenize(text)

    # Initialize Lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Get Stopwords and Punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    # Apply stopwords, punctuation removal, and lemmatization
    words = [
        lemmatizer.lemmatize(word.lower(), get_wordnet_pos(pos))
        for word, pos in pos_tag(tokenized_words)
        if word not in stop_words and word not in punctuation
    ]
    
    word_counts = Counter(words)
    
    # Fix: Ensure vocabulary indexing starts from 0
    vocab = {word: i for i, (word, _) in enumerate(word_counts.most_common())}
    vocab['<UNK>'] = len(vocab)  # Last index
    vocab_size = len(vocab)

    sequences = []
    for i in range(len(words) - sequence_length):
        seq = words[i: i + sequence_length]
        sequences.append([vocab.get(word, vocab['<UNK>']) for word in seq])

    return sequences, vocab, vocab_size



In [75]:
# Custom Dataset class
class TextDataset(Dataset):
    def __init__(self, sequences):
        self.x = torch.tensor([seq[:-1] for seq in sequences], dtype=torch.long)
        self.y = torch.tensor([seq[-1] for seq in sequences], dtype=torch.long)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


In [76]:
# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take last output of LSTM
        return x

In [77]:
# Training Function
def train_model(model, dataloader, vocab_size, epochs=25, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()
            
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")


In [78]:

# Prediction Function
def predict(model, text, vocab, sequence_length=5):
    model.eval()

    # Tokenize input text
    words = word_tokenize(text.lower())

    #  Remove words not in vocab
    filtered_words = [word for word in words if word in vocab]
    if not filtered_words:
       print("Error: None of the words in input_text exist in the vocabulary!")
       return "<UNK>"

    # Convert words to indices
    sequence = [vocab.get(word, vocab['<UNK>']) for word in filtered_words][-sequence_length:]
    
    
    # Debugging outputs
    print("Tokenized Input:", words)
    print("Filtered Words:", filtered_words)
    print("Mapped Sequence:", sequence)
    
    input_tensor = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        print(output.shape)
        predicted_idx = torch.argmax(output, dim=1).item()
    
    inv_vocab = {idx: word for word, idx in vocab.items()}
    return inv_vocab.get(predicted_idx, '<UNK>')


In [79]:
# Main Execution
filepath = r"C:\Users\abdul.muhmin\Downloads\Sherlock_Holmes.txt"
text = load_data(filepath)
sequences, vocab, vocab_size = preprocess_text(text)
dataset = TextDataset(sequences)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [80]:
# Initialize and train model
model = LSTMModel(vocab_size)
train_model(model, dataloader, vocab_size)


Epoch 1, Loss: 7.7867
Epoch 2, Loss: 6.9681
Epoch 3, Loss: 5.8078
Epoch 4, Loss: 4.6313
Epoch 5, Loss: 3.6731
Epoch 6, Loss: 2.9239
Epoch 7, Loss: 2.3086
Epoch 8, Loss: 1.7866
Epoch 9, Loss: 1.3405
Epoch 10, Loss: 0.9693
Epoch 11, Loss: 0.6691
Epoch 12, Loss: 0.4396
Epoch 13, Loss: 0.2727
Epoch 14, Loss: 0.1634
Epoch 15, Loss: 0.0981
Epoch 16, Loss: 0.0641
Epoch 17, Loss: 0.0460
Epoch 18, Loss: 0.0364
Epoch 19, Loss: 0.0332
Epoch 20, Loss: 0.0318
Epoch 21, Loss: 0.0351
Epoch 22, Loss: 0.0247
Epoch 23, Loss: 0.0178
Epoch 24, Loss: 0.0450
Epoch 25, Loss: 0.0876


In [89]:
# Test Prediction
input_text = "THE ADVENTURES OF SHERLOCK "
predicted_word = predict(model, input_text, vocab)
print(f"Predicted completion: {predicted_word}")


Tokenized Input: ['the', 'adventures', 'of', 'sherlock']
Filtered Words: ['sherlock']
Mapped Sequence: [57]
torch.Size([1, 6705])
Predicted completion: holmes


In [91]:
# Test Prediction
input_text = "The Adventure of the Noble "
predicted_word = predict(model, input_text, vocab)
print(f"Predicted completion: {predicted_word}")


Tokenized Input: ['the', 'adventure', 'of', 'the', 'noble']
Filtered Words: ['adventure', 'noble']
Mapped Sequence: [349, 558]
torch.Size([1, 6705])
Predicted completion: bachelor
